In [ ]:
import torch
from torch import nn
from torch.utils.checkpoint import checkpoint
from model import StocksPredictionModel
from stocks_dataset import StocksDataSet

In [ ]:
#%run Model.ipynb
# %run "../Machine Learning Data Gathering/Scripts/Moving Average + Bollinger bands.ipynb"
#%run StocksDataSet.ipynb

In [ ]:
#export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"
# del os.environ["PYTORCH_CUDA_ALLOC_CONF"]

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

# Training the model

In [ ]:
def train_model(ds, 
                input_size = 16, 
                hidden_size = 256, 
                num_layers = 1, 
                num_classes = 16, 
                batch_first = True, 
                epochs_number = 5000, 
                device = device, 
                learning_rate = 0.001):
    
    model = StocksPredictionModel(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes = num_classes, batch_first)
    custom_ds = ds
    
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(params = model.parameters(), lr=learning_rate)
    epochs = epochs_number
    
    for epoch in range(epochs):
        
        for X_train, X_test, y_train, y_test in custom_ds:
            if X_train is not None:
                X_train = torch.Tensor(X_train).squeeze(0)
                model.train()
                model = model.to(device)
                X_train.to(device)
        
                y_pred = model(X_train)
            
            
                optimizer.zero_grad()
                y_pred = y_pred.to(device)
                y_train = y_train.to(device)
                loss = loss_fn(y_pred, y_train)
                loss = loss.to(device)
            
                loss.backward()
            
                optimizer.step()
            
                model.eval()
                
            
                with torch.inference_mode():
                
                    test_pred = model(X_test)
                    test_pred.to(device)
                    y_test = y_test.to(device)
                    test_loss = loss_fn(test_pred, y_test)
                    if epoch == epochs - 1 or epoch % 100 == 0:
                        print(f"Epoch number: {epoch}")
                        print(f"Test Loss is: {test_loss}")
                        print(f"Train Loss is: {loss}")
            
    save_with_kwargs(model, epochs_number, learning_rate, ds.preparationType.value)

# Saving the Model

In [ ]:
def save_with_kwargs(model, epochs, lr, split_type):
    torch.save([model.kwargs, model.state_dict()], f"./Models/{epochs}epochs_{model.modelType.value}_{lr}lr_{split_type}.pth")

# Training process testing

In [ ]:
test = StocksDataSet("../Custom LSTM Model/Data/Stocks Data")
our_model = train_model(test, epochs_number=500, learning_rate=0.001, hidden_size = 128, num_classes=30)

In [ ]:
save_with_kwargs(our_model, name = "16_args_with_test_split_stand_normalize_500epochs_lr001_simplemodel")